# List of Exposures with Hologram


- work with Weakly_2022_09
- use jupyter kernel LSST



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/03/22

- update : 2022/05/05
- last update : 2022/05/19 : check if 2022/05/04 where already ingested


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/atmospec 	setup
eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_09/conda/envs/lsst-scipipe-2.0.0/eups 	setup
rapid_analysis        LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/rapid_analysis 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


   g64fc59b30a+e1dba0a6f6 	w_latest w_2022_09 current setup


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [4]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [5]:
import lsst.daf.butler as dafButler

In [6]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [7]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [8]:
for count, info in enumerate(registry.queryDimensionRecords('exposure')):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [9]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2021021600208,AT_O_20210216_000208,20210216,208,science,HD 50896,empty~empty,24.314073,2.0,103.384002,-23.973448,-180.0
1,2021021600163,AT_O_20210216_000163,20210216,163,science,HD 50896,RG610~empty,14.580611,3.0,103.404029,-23.92939,-208.621712
2,2021021600070,AT_O_20210216_000070,20210216,70,bias,azel_target,empty~empty,None,0.0,None,None,None
3,2021021600173,AT_O_20210216_000173,20210216,173,science,HD 50896,RG610~empty,8.82931,2.0,103.395986,-23.927038,-208.621712
4,2021021600001,AT_O_20210216_000001,20210216,1,bias,azel_target,empty~empty,None,0.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
15821,2022050400606,AT_O_20220504_000606,20220504,606,science,LATISS_E6A_00000001,SDSSi~empty,14.989754,30.0,220.816016,-44.779265,0.00031
15822,2022050400607,AT_O_20220504_000607,20220504,607,science,LATISS_E6A_00000001,SDSSg~empty,15.021553,30.0,220.815965,-44.779219,0.000356
15823,2022050400608,AT_O_20220504_000608,20220504,608,science,LATISS_E6A_00000001,SDSSg~empty,15.047233,30.0,220.815981,-44.779269,0.000528
15824,2022050400609,AT_O_20220504_000609,20220504,609,science,LATISS_E6A_00000001,SDSSr~empty,15.080195,30.0,220.816128,-44.779252,0.000063


## Selection of science

In [10]:
df_science = df_exposure[df_exposure.type == 'science']

In [11]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_1723/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [12]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
15666,2022050400451,AT_O_20220504_000451,20220504,451,science,LATISS_E6A_00000019,SDSSi~empty,34.958386,30.0,221.803725,-44.873005,359.998891
15667,2022050400452,AT_O_20220504_000452,20220504,452,science,LATISS_E6A_00000087,SDSSi~empty,34.7916,30.0,221.549013,-45.528988,359.999438
15668,2022050400453,AT_O_20220504_000453,20220504,453,science,LATISS_E6A_00000087,SDSSi~empty,34.697292,30.0,221.54897,-45.529006,359.998759
15669,2022050400454,AT_O_20220504_000454,20220504,454,science,LATISS_E6A_00000087,SDSSg~empty,34.583117,30.0,221.549034,-45.529022,359.99874
15670,2022050400455,AT_O_20220504_000455,20220504,455,science,LATISS_E6A_00000087,SDSSg~empty,34.4889,30.0,221.548979,-45.529032,359.998766
15671,2022050400456,AT_O_20220504_000456,20220504,456,science,LATISS_E6A_00000087,SDSSr~empty,34.374794,30.0,221.54897,-45.529024,359.998877
15672,2022050400457,AT_O_20220504_000457,20220504,457,science,LATISS_E6A_00000087,SDSSr~empty,34.280725,30.0,221.548986,-45.529013,359.999867
15673,2022050400458,AT_O_20220504_000458,20220504,458,science,LATISS_E6A_00000004,SDSSr~empty,33.76664,30.0,221.21313,-44.778758,359.998351
15674,2022050400459,AT_O_20220504_000459,20220504,459,science,LATISS_E6A_00000004,SDSSr~empty,33.671172,30.0,221.213024,-44.779265,359.998848
15675,2022050400460,AT_O_20220504_000460,20220504,460,science,LATISS_E6A_00000004,SDSSg~empty,33.555471,30.0,221.213016,-44.779272,359.999189


## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503

In [13]:
date_sel = 20220504

In [14]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [15]:
len(df_science_selected)

280

# List of filters

In [16]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~empty', 'SDSSr~empty', 'SDSSg~empty', 'SDSSi~empty'],
      dtype=object)

In [17]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array([], dtype=float64)

## Selection of filter

- But now we have a loop on filters

In [18]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [19]:
pd.set_option('display.max_rows', None)

In [20]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

In [21]:
len(all_def_science_selected_final)

0

In [22]:
all_def_science_selected_final[0]

IndexError: list index out of range

In [ ]:
all_def_science_selected_final[1]

In [ ]:
all_def_science_selected_final[2]

In [ ]:
all_def_science_selected_final[3]

In [ ]:
all_def_science_selected_final[4]